# Segmentation and Clustering of Toronto Neighborhood

In [59]:
#Loading all required libraries
import numpy as np
import pandas as pd
import os
import requests

import geocoder
from geopy.geocoders import Nominatim

import folium
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [60]:
#Setting up working directory for saving files
os.chdir('C:\Sam\Per\IDM\Python\Coursera\Capstone')

###### Question No. 1

In [61]:
#Reading the data from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header = 0)[0]

#Printing the head of data
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [62]:
#Rename postal code column
df.rename(columns = {'Postal code' : 'PostalCode'}, inplace = True)

#Delete rows with no borough assigned
df = df[~(df['Borough'] == 'Not assigned')].reset_index(drop = True)

#Seperating neighborhood with comma rather than backslash
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',')

###### Shape and Head rows of the data

In [63]:
#Checking the dimension of data
df.shape

(103, 3)

In [64]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


###### Could not read the geographical data from google as while loop was never ending
###### Question 2

In [65]:
#Read geographical coordinate data
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True)
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


###### Display the first 10 records after adding Latitude and Longitude of each Postal Code

In [66]:
#Merge geographical data with postal data
df = pd.merge(df, geo, on = 'PostalCode', how = 'left')
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937


###### Question 3

In [100]:
#Getting Latitude and Longitude of Toronto
address = 'Toronto, Ontario'

geoloc = Nominatim(user_agent= 'Tor_explorer')
location = geoloc.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geolocation coordinated of Toronto are {} and {}'.format(latitude, longitude))

The geolocation coordinated of Toronto are 43.6534817 and -79.3839347


###### Create a map of Toronto with neighborhoods on top of it

In [68]:

map_tor = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lon, bor, nhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(nhood, bor)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_tor)
        
map_tor

###### Define Foursquare Credentials and Version

In [71]:
client_id = '05EQ5TY40Z3VPDZF0EAA3SVNZ3QG4DQVHZBTRMFETCT53PPD'
client_secret = '44QA1XXKZQTVNUQ0VHH1B5NACSZ1CLXBNJCS2QQ4NSCJGA3N'
version = '20200430'
limit = 100

###### Function to explore nearby venues and create dataframe with details including venure category

In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    venues_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        #Foursquare api call to explore venues
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
        .format(client_id, client_secret, version, lat, lng, radius, limit)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #Appending the venue details to a list
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    #Creating dataframe and copying the venue details
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [73]:
#Call function to get nearby venues
df_venues = getNearbyVenues(df['Neighborhood'], df['Latitude'], df['Longitude'])
df_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [101]:
# Checking the shape, number of unique neighborhood and number of unique venue category
print(df_venues.shape)
print(df_venues['Neighborhood'].nunique())
print(df_venues['Venue Category'].nunique())

(2151, 7)
93
267


###### Analyze Each Neighborhood

In [75]:
#One hot encoding
df_ohe = pd.get_dummies(df_venues[['Venue Category']], prefix = '', prefix_sep = '')

# add neighborhood column back to dataframe
df_ohe['Neighborhood_area'] = df_venues['Neighborhood']

# move neighborhood column to the first column
fixed_col = [df_ohe.columns[-1]] + list(df_ohe.columns[:-1])
df_ohe = df_ohe[fixed_col]

df_ohe.head()

,Neighborhood_area,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
#Check the shape of new dataframe
df_ohe.shape

(2151, 268)

###### Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [ ]:
df_toronto = df_ohe.groupby('Neighborhood_area').mean().reset_index()
df_toronto.head()

###### Check the new shape

In [78]:
df_toronto.shape

(93, 268)

###### Function to sort the venues in descending order

In [79]:
def return_most_common_venues(row, num_top_venues):
    row_categories_sorted = row.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
nhood_venues_sort = pd.DataFrame(columns = columns)
nhood_venues_sort['Neighborhood'] = df_toronto['Neighborhood_area']

#Call the function to sort venues in descending order
for ind in np.arange(df_toronto.shape[0]):
    nhood_venues_sort.iloc[ind, 1:] = return_most_common_venues(df_toronto.iloc[ind, 1:], num_top_venues)

nhood_venues_sort.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood , Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Dance Studio,Coffee Shop,Pub,Sandwich Place,Skating Rink,Gym,Antique Shop
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket,Sushi Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Greek Restaurant,Thai Restaurant,Cosmetics Shop,Pharmacy,Pizza Place


###### Clustering using KMeans

In [89]:
# set number of clusters
kclusters = 7

df_clus = df_toronto.drop('Neighborhood_area', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(df_clus)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 1, 1, 1, 1, 1, 2, 2])

In [91]:
# add clustering labels
nhood_venues_sort.insert(0, 'Cluster', kmeans.labels_)

df_final = df

#Merge to add both neighborhood details and venue details
df_final = pd.merge(df_final, nhood_venues_sort, on = 'Neighborhood', how = 'left')

#Drop any neighborhood which does not have any venue
df_final.dropna(axis = 0, inplace = True)

#Convert back clusters to integer type
df_final['Cluster'] = df_final['Cluster'].astype(np.int)

df_final.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944,2,Park,Smoke Shop,Pool,River,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Gay Bar,Grocery Store,Hotel,Mediterranean Restaurant,Men's Store
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558,2,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509,3,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Filipino Restaurant
102,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.628841,-79.520999,2,Hardware Store,Tanning Salon,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Burrito Place,Burger Joint,Sandwich Place,Bakery


In [103]:
# create map
cluster_map = folium.Map(location = [latitude, longitude], zoom_start = 10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lng, nhood, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood'], df_final['Cluster']):
    label = folium.Popup(str(nhood) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity= 0.7).add_to(cluster_map)

cluster_map

###### Check each cluster

###### Cluster 1

In [93]:
df_final.loc[df_final['Cluster'] == 0, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Weston,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
1,"Moore Park , Summerhill East",Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


###### Cluster 2

In [94]:
df_final.loc[df_final['Cluster'] == 1, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park , Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
1,"Lawrence Manor , Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Shoe Store,Sporting Goods Shop,Miscellaneous Shop,Arts & Crafts Store,Boutique
2,"Queen's Park , Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Park,Bar,Beer Bar,Sandwich Place,Burger Joint,Burrito Place
3,Don Mills,Gym,Asian Restaurant,Beer Store,Restaurant,Japanese Restaurant,Coffee Shop,Café,Discount Store,Supermarket,Baseball Field
4,"Garden District , Ryerson",Clothing Store,Coffee Shop,Café,Japanese Restaurant,Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Theater
5,Don Mills,Gym,Asian Restaurant,Beer Store,Restaurant,Japanese Restaurant,Coffee Shop,Café,Discount Store,Supermarket,Baseball Field
6,St. James Town,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Seafood Restaurant,Theater,Creperie,Department Store,Clothing Store
7,"Eringate , Bloordale Gardens , Old Burnhamthor...",Pizza Place,Beer Store,Convenience Store,Coffee Shop,Cosmetics Shop,Café,Shopping Plaza,Liquor Store,Pet Store,General Travel
8,Berczy Park,Coffee Shop,Cocktail Bar,Café,Restaurant,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Lounge,Sporting Goods Shop
9,Woburn,Coffee Shop,Korean Restaurant,Convenience Store,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


###### Cluster 3

In [95]:
df_final.loc[df_final['Cluster'] == 2, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Victoria Village,Pizza Place,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Intersection,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Dessert Shop
1,"Malvern , Rouge",Fast Food Restaurant,Print Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
2,"Parkview Hill , Woodbine Gardens",Pizza Place,Gastropub,Pharmacy,Gym / Fitness Center,Breakfast Spot,Fast Food Restaurant,Bank,Intersection,Athletics & Sports,Pet Store
3,Glencairn,Metro Station,Pizza Place,Park,Japanese Restaurant,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,"Rouge Hill , Port Union , Highland Creek",Construction & Landscaping,History Museum,Bar,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
5,Woodbine Heights,Park,Cosmetics Shop,Beer Store,Diner,Dance Studio,Athletics & Sports,Curling Ice,Skating Rink,Video Store,Pharmacy
6,Humewood-Cedarvale,Field,Dog Run,Hockey Arena,Trail,Donut Shop,Diner,Discount Store,Distribution Center,Doner Restaurant,Yoga Studio
7,"Guildwood , Morningside , West Hill",Rental Car Location,Breakfast Spot,Medical Center,Intersection,Electronics Store,Mexican Restaurant,Bank,Yoga Studio,Doner Restaurant,Distribution Center
8,The Beaches,Health Food Store,Neighborhood,Pub,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
9,Thorncliffe Park,Indian Restaurant,Yoga Studio,Park,Sandwich Place,Burger Joint,Restaurant,Coffee Shop,Pizza Place,Pharmacy,Discount Store


###### Cluster 4

In [96]:
df_final.loc[df_final['Cluster'] == 3, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Humberlea , Emery",Paper / Office Supplies Store,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
1,"Old Mill South , King's Mill Park , Sunnylea ,...",Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Filipino Restaurant


###### Cluster 5

In [97]:
df_final.loc[df_final['Cluster'] == 4, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough Village,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


###### Cluster 6

In [98]:
df_final.loc[df_final['Cluster'] == 5, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Eastern European Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
1,Caledonia-Fairbanks,Park,Women's Store,Spa,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,East Toronto,Park,Convenience Store,Coffee Shop,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
3,York Mills West,Park,Bank,Convenience Store,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Milliken , Agincourt North , Steeles East , L'...",Playground,Park,Coffee Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
5,Rosedale,Park,Playground,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


###### Cluster 7

In [99]:
df_final.loc[df_final['Cluster'] == 6, df_final.columns[[2] + list(range(6, df_final.shape[1]))]].reset_index(drop = True)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roselawn,Garden,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
